# Scraping apps reviews from Google play store and perform sentiment analysis

In [1]:
!pip install google-play-scraper

Defaulting to user installation because normal site-packages is not writeable

You should consider upgrading via the 'c:\program files\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python39\python.exe -m pip install --upgrade pip' command.


In [3]:
from google_play_scraper import Sort, reviews_all, reviews, app
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
nltk.download('vader_lexicon')
import seaborn as sns
plt.style.use('ggplot')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\iamlo/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
result, continuation_token = reviews(
    'com.miniclip.eightballpool',
    #sleep_milliseconds=0,
    lang = 'en',
    country='us',
    sort = Sort.NEWEST,
    count = 500,
    filter_score_with=None
)

In [5]:
scrapeddata = pd.DataFrame(np.array(result), columns=['review'])
scrapeddata = scrapeddata.join(pd.DataFrame(scrapeddata.pop('review').tolist()))
scrapeddata.head()

reviewId          userName  \
0  591e5815-19e6-48dc-82f3-adcead5133f8   Rachmat Wahyudi   
1  174d9d57-f2eb-4b86-b0cd-1fceb943f593         EVIL King   
2  fee81723-3de0-4086-a17a-fed5befa106e       Saif Abbasi   
3  a7187cfa-4975-478d-b5fe-3f93d9062d10  Karren Laurencio   
4  9aa7983e-7864-4e97-96ce-e0403d4bf32b       Betty White   

                                           userImage  \
0  https://play-lh.googleusercontent.com/a-/ACB-R...   
1  https://play-lh.googleusercontent.com/a-/ACB-R...   
2  https://play-lh.googleusercontent.com/a-/ACB-R...   
3  https://play-lh.googleusercontent.com/a/AGNmyx...   
4  https://play-lh.googleusercontent.com/a-/ACB-R...   

                                             content  score  thumbsUpCount  \
0                                                 0k      5              0   
1  i am very happy 😁😁..to play this game this gam...      5              0   
2                                          nice game      5              0   
3  Its verry fun i keep playing every time i have...      5              0   
4  8 Ball on this site. the only thing I don't ca...      5              0   

  reviewCreatedVersion                  at replyContent repliedAt  
0               5.12.0 2023-03-14 17:18:14         None       NaT  
1               5.12.0 2023-03-14 17:13:06         None       NaT  
2               5.12.0 2023-03-14 17:11:19         None       NaT  
3               5.12.0 2023-03-14 17:09:19         None       NaT  
4               5.12.0 2023-03-14 17:07:40         None       NaT

In [6]:
len(scrapeddata.index)

500

In [7]:
scrapeddata[['userName','content', 'score', 'at']].head()

userName                                            content  score  \
0   Rachmat Wahyudi                                                 0k      5   
1         EVIL King  i am very happy 😁😁..to play this game this gam...      5   
2       Saif Abbasi                                          nice game      5   
3  Karren Laurencio  Its verry fun i keep playing every time i have...      5   
4       Betty White  8 Ball on this site. the only thing I don't ca...      5   

                   at  
0 2023-03-14 17:18:14  
1 2023-03-14 17:13:06  
2 2023-03-14 17:11:19  
3 2023-03-14 17:09:19  
4 2023-03-14 17:07:40

In [8]:
scrapeddata1 = scrapeddata[['userName','content', 'score', 'at']]
sorteddata = scrapeddata1.sort_values(by = 'at', ascending=False)
sorteddata.head()

userName                                            content  score  \
0   Rachmat Wahyudi                                                 0k      5   
1         EVIL King  i am very happy 😁😁..to play this game this gam...      5   
2       Saif Abbasi                                          nice game      5   
3  Karren Laurencio  Its verry fun i keep playing every time i have...      5   
4       Betty White  8 Ball on this site. the only thing I don't ca...      5   

                   at  
0 2023-03-14 17:18:14  
1 2023-03-14 17:13:06  
2 2023-03-14 17:11:19  
3 2023-03-14 17:09:19  
4 2023-03-14 17:07:40

In [9]:
sorteddata.to_csv("8ball_data.csv", index = False)

In [10]:
dataset =  pd.read_csv("C:/Users/iamlo/nitesh/src/Jupyter Notebook/DA/8ball_data.csv")

In [11]:
dataset.head()

userName                                            content  score  \
0   Rachmat Wahyudi                                                 0k      5   
1         EVIL King  i am very happy 😁😁..to play this game this gam...      5   
2       Saif Abbasi                                          nice game      5   
3  Karren Laurencio  Its verry fun i keep playing every time i have...      5   
4       Betty White  8 Ball on this site. the only thing I don't ca...      5   

                    at  
0  2023-03-14 17:18:14  
1  2023-03-14 17:13:06  
2  2023-03-14 17:11:19  
3  2023-03-14 17:09:19  
4  2023-03-14 17:07:40

In [12]:
dataset['score'].value_counts()

5    316
1    111
4     37
3     19
2     17
Name: score, dtype: int64

In [13]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
sia = SentimentIntensityAnalyzer()

In [14]:
res={}
for i,row in tqdm(dataset.iterrows(), total=len(dataset)):
  text =  row['content']
  myid = row['userName']
  res[myid] = sia.polarity_scores(text)

  0%|          | 0/500 [00:00<?, ?it/s]

In [15]:
pd.DataFrame(res).T

neg    neu    pos  compound
Rachmat Wahyudi         0.000  1.000  0.000    0.0000
EVIL King               0.000  0.576  0.424    0.8610
Saif Abbasi             0.000  0.263  0.737    0.4215
Karren Laurencio        0.000  0.488  0.512    0.8126
Betty White             0.162  0.795  0.043   -0.6727
...                       ...    ...    ...       ...
RAHUL SINGH             0.434  0.566  0.000   -0.3182
Prashanth.P Khatavakar  0.222  0.568  0.211   -0.0324
Rico Refujia            0.000  0.000  1.000    0.4215
Ravy love               0.000  1.000  0.000    0.0000
Last Romeo              0.000  1.000  0.000    0.0000

[496 rows x 4 columns]

In [16]:
result, continuation_token = reviews(
    'com.ludo.king',
    #sleep_milliseconds=0,
    lang = 'en',
    country='us',
    sort = Sort.NEWEST,
    count = 600,
    filter_score_with=None
)

In [17]:
scrapeddata2 = pd.DataFrame(np.array(result), columns=['review'])
scrapeddata2 = scrapeddata2.join(pd.DataFrame(scrapeddata2.pop('review').tolist()))
scrapeddata2.head()

reviewId                 userName  \
0  e60cba7b-1e3c-4c18-8d34-5266fab5935a              Sahil Kumar   
1  1df207d6-7621-4a62-b684-98a6a2a6e2ad             Rupesh kumar   
2  3a5a83c9-f731-4d15-a6ac-7549eaeda3f3  defender star Khongiong   
3  4954de29-ac26-4225-b15a-e03a128e0331              Nobolila Tv   
4  ef289e97-bea2-49e5-8a78-45d2af2f80c3             Mohit Madhad   

                                           userImage  \
0  https://play-lh.googleusercontent.com/a/AGNmyx...   
1  https://play-lh.googleusercontent.com/a/AGNmyx...   
2  https://play-lh.googleusercontent.com/a/AGNmyx...   
3  https://play-lh.googleusercontent.com/a-/ACB-R...   
4  https://play-lh.googleusercontent.com/a/AGNmyx...   

                                content  score  thumbsUpCount  \
0                     Africa gayrg Ahwd      5              0   
1  Roshanisunar Rekhansunar Roshansunar      5              0   
2                     Wow nice good 👍😉🙂      5              0   
3                             Good game      5              0   
4                              ravinser      1              0   

  reviewCreatedVersion                  at  \
0            7.7.0.243 2023-03-14 17:11:45   
1                  2.4 2023-03-14 17:07:37   
2                 None 2023-03-14 17:01:37   
3                  4.3 2023-03-14 16:59:21   
4                 None 2023-03-14 16:58:31   

                             replyContent           repliedAt  
0                                    None                 NaT  
1                                    None                 NaT  
2                                    None                 NaT  
3  Thank you so much for your feedback :) 2023-03-15 16:00:23  
4                                    None                 NaT

In [18]:
len(scrapeddata2.index)

600

In [19]:
scrapeddata2[['userName','content', 'score', 'at']].head()

userName                               content  score  \
0              Sahil Kumar                     Africa gayrg Ahwd      5   
1             Rupesh kumar  Roshanisunar Rekhansunar Roshansunar      5   
2  defender star Khongiong                     Wow nice good 👍😉🙂      5   
3              Nobolila Tv                             Good game      5   
4             Mohit Madhad                              ravinser      1   

                   at  
0 2023-03-14 17:11:45  
1 2023-03-14 17:07:37  
2 2023-03-14 17:01:37  
3 2023-03-14 16:59:21  
4 2023-03-14 16:58:31

In [20]:
S2 = scrapeddata2[['userName','content', 'score', 'at']]
S = S2.sort_values(by = 'at', ascending=False)
S.head()

userName                               content  score  \
0              Sahil Kumar                     Africa gayrg Ahwd      5   
1             Rupesh kumar  Roshanisunar Rekhansunar Roshansunar      5   
2  defender star Khongiong                     Wow nice good 👍😉🙂      5   
3              Nobolila Tv                             Good game      5   
4             Mohit Madhad                              ravinser      1   

                   at  
0 2023-03-14 17:11:45  
1 2023-03-14 17:07:37  
2 2023-03-14 17:01:37  
3 2023-03-14 16:59:21  
4 2023-03-14 16:58:31

In [21]:
S['score'].value_counts()

5    361
1    137
4     49
3     31
2     22
Name: score, dtype: int64

In [22]:
res={}
for i,row in tqdm(S.iterrows(), total=len(S)):
  text =  row['content']
  myid = row['userName']
  res[myid] = sia.polarity_scores(text)

  0%|          | 0/600 [00:00<?, ?it/s]

In [23]:
pd.DataFrame(res).T

neg    neu    pos  compound
Sahil Kumar              0.0  1.000  0.000    0.0000
Rupesh kumar             0.0  1.000  0.000    0.0000
defender star Khongiong  0.0  0.095  0.905    0.8591
Nobolila Tv              0.0  0.256  0.744    0.4404
Mohit Madhad             0.0  1.000  0.000    0.0000
...                      ...    ...    ...       ...
Shohag Patowary          0.0  1.000  0.000    0.0000
Endal Kumar              0.0  0.000  1.000    0.5994
Uttm Das                 0.0  1.000  0.000    0.0000
Jeetu Keer               0.0  0.000  0.000    0.0000
Meenu Yadav              0.0  0.238  0.762    0.4927

[596 rows x 4 columns]